<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [3]:
import folium
import wget
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [5]:
# Download and read the `spacex_launch_geo.csv`
import folium
import pandas as pd

# Load the launch site dataset
spacex_df = pd.read_csv("spacex_launch_geo.csv")
spacex_df.head()
# Initial map centered around Florida (approximate center of launch sites)
site_map = folium.Map(location=[28.5, -80.5], zoom_start=5)

# Add launch site markers
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

# Display map
site_map



Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
# Select only the relevant columns
sites_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Display the unique sites and their coordinates
sites_df.groupby('Launch Site').first().reset_index()


,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [10]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [11]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [12]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
import folium
from folium.plugins import MarkerCluster

# Step 1: Create a new column for marker colors based on launch outcome
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Step 2: Initialize a map centered around Cape Canaveral
site_map = folium.Map(location=[28.5, -80.5], zoom_start=5)

# Step 3: Create a MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Step 4: Add markers for each launch
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {row['class']}",
        icon=folium.Icon(color=row['marker_color'], icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Step 5: Display map
site_map


In [13]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_color
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895,green
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895,green
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895,green
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820,green
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820,green
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820,red
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820,red
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820,red
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820,green
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [17]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Example: Pick one launch site (CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Example: Marked coordinates of closest coastline (from MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance
distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon, coastline_lat, coastline_lon
)

print(f"Distance to coastline: {distance_coastline:.2f} km")

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site: CCAFS SLC-40",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa"),
).add_to(site_map)

# Add a marker for the coastline point
folium.Marker(
    [coastline_lat, coastline_lon],
    popup=f"Coastline Point\nDistance: {distance_coastline:.2f} km",
    icon=folium.Icon(color="orange", icon="flag"),
).add_to(site_map)

# Optionally, draw a line between launch site and coastline
folium.PolyLine(
    [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color="blue",
    weight=2,
    dash_array="5, 5"
).add_to(site_map)

# Display the map
site_map


Distance to coastline: 0.51 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [18]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
# Example launch site (CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Example closest coastline coordinate (from MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon, coastline_lat, coastline_lon
)

# Add marker for the coastline point
folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="orange", icon="flag")
).add_to(site_map)

# Add distance label as a DivIcon
distance_marker = folium.Marker(
    [ (launch_site_lat + coastline_lat)/2 , (launch_site_lon + coastline_lon)/2 ],  # midpoint of line
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' 
             % "{:10.2f} KM".format(distance_coastline),
    )
).add_to(site_map)

# Draw a line between launch site and coastline
folium.PolyLine(
    [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color="blue", weight=2, dash_array="5, 5"
).add_to(site_map)

# Show map
site_map


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [19]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# Example launch site (CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Example closest coastline coordinate (from MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Create a PolyLine between launch site and coastline
lines = folium.PolyLine(
    locations=[
        [launch_site_lat, launch_site_lon],
        [coastline_lat, coastline_lon]
    ],
    weight=2,  # line thickness
    color="blue",  # optional: set color
    dash_array="5,5"  # optional: dashed line
)

# Add to the map
site_map.add_child(lines)

# Display map
site_map



Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [24]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Example: Launch site (KSC LC-39A)
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Example: Closest city (from MousePosition)
city_lat = 28.6100
city_lon = -80.8070

# Calculate distance
distance_city = calculate_distance(
    launch_site_lat, launch_site_lon, city_lat, city_lon
)

# Add marker for the city
folium.Marker(
    [city_lat, city_lon],
    popup="Closest City",
    icon=folium.Icon(color="purple", icon="building", prefix="fa")
).add_to(site_map)

# Add distance label at midpoint using DivIcon
folium.Marker(
    [(launch_site_lat + city_lat)/2, (launch_site_lon + city_lon)/2],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#800080;"><b>%s</b></div>' 
             % "{:10.2f} KM".format(distance_city),
    )
).add_to(site_map)

# Draw a line between launch site and city
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    color="purple",
    weight=2,
    dash_array="5,5"
).add_to(site_map)



import folium
from folium.features import DivIcon

# Example launch site: KSC LC-39A
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Example nearby locations (replace with actual closest coordinates)
locations = {
    "Closest City": (28.6100, -80.8070),
    "Closest Railway": (28.5800, -80.7000),
    "Closest Highway": (28.5750, -80.6800),
    "Closest Coastline": (28.5600, -80.6200)
}

# Function to calculate distance in KM (Haversine formula)
from math import radians, cos, sin, asin, sqrt
def calculate_distance(lat1, lon1, lat2, lon2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of Earth in km
    return c * r

# Create a map centered on the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add launch site marker
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site: KSC LC-39A",
    icon=folium.Icon(color="red", icon="rocket", prefix="fa")
).add_to(site_map)

# Add markers and lines for each nearby location
for name, (lat, lon) in locations.items():
    # Distance
    distance = calculate_distance(launch_site_lat, launch_site_lon, lat, lon)
    
    # Marker for location
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color="purple", icon="info-sign")
    ).add_to(site_map)
    
    # Add distance label at midpoint
    folium.Marker(
        [(launch_site_lat + lat)/2, (launch_site_lon + lon)/2],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#800080;"><b>{distance:.2f} KM</b></div>'
        )
    ).add_to(site_map)
    
    # Draw polyline between launch site and location
    folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], [lat, lon]],
        color="purple",
        weight=2,
        dash_array="5,5"
    ).add_to(site_map)

site_map



After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.



Are launch sites in close proximity to railways?
Distance: 5.24 KM
Yes, the launch site is quite close to the railway. This is typical for transporting rocket components, fuel, and other heavy cargo to the site efficiently.
Are launch sites in close proximity to highways?
Distance: 3.24 KM
Yes, very close. Highways are essential for crew access, equipment transport, and emergency logistics.
Are launch sites in close proximity to coastline?
Distance: 3.01 KM
Yes, very close. Proximity to the ocean allows rockets to launch safely eastward over water and facilitates booster recovery operations.
Do launch sites keep certain distance away from cities?
Distance: 16.16 KM
Yes, launch sites are deliberately kept at a safe distance from populated areas. This minimizes the risk to human life and property in the event of launch anomalies or debris fallout


In [26]:
for name, (lat, lon) in locations.items():
    # Calculate distance
    distance = calculate_distance(launch_site_lat, launch_site_lon, lat, lon)
    
    # Print the distance
    print(f"Distance from Launch Site to {name}: {distance:.2f} KM")
    
    # Marker for location
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color="purple", icon="info-sign")
    ).add_to(site_map)
    
    # Add distance label at midpoint
    folium.Marker(
        [(launch_site_lat + lat)/2, (launch_site_lon + lon)/2],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#800080;"><b>{distance:.2f} KM</b></div>'
        )
    ).add_to(site_map)
    
    # Draw polyline between launch site and location
    folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], [lat, lon]],
        color="purple",
        weight=2,
        dash_array="5,5"
    ).add_to(site_map)


Distance from Launch Site to Closest City: 16.16 KM
Distance from Launch Site to Closest Railway: 5.24 KM
Distance from Launch Site to Closest Highway: 3.24 KM
Distance from Launch Site to Closest Coastline: 3.01 KM


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
